# Optimizing Model Parameters

지금까지 모델 만들었고, 데이터 불러왔으니 학습을 시켜보자
우리 데이터로 파라메터 최적화를 하여 모델을 입증하고 테스트하는 거야
모델을 학습하는 건 반복적인 처리인데, 각 반복에서 모델은 출력에 대한 예측을 하고 
예측에 대한 오차(loss)를 계산해, 오차를 각 파라미터를 기준 변수로 해서 미분한 값들을 모은다.
그리고 이 파라미터들을 경사 하강법 이용하여 최적화한다.


## Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%
100.0%
100.0%
100.0%


## Hyperparameters

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5